In [ ]:
# Importa as bibliotecas

import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

In [173]:
ticker = "AAPL"

# Baixar dados
data = yf.download(ticker, start = "2010-01-01", end = "2025-01-01")

# Calcular a média móvel de 50 dias
data["SMA50"] = data["Adj Close"].rolling(window = 50).mean()

def detectar_bandeiras(data, janela_haste = 20, janela_bandeira = 15, threshold_haste = 0.05, dias_apos_bandeira = 5):
    bandeiras = []
    movimentos = []
    i = janela_haste + janela_bandeira + 50  # Começar a busca após dados suficientes para janela de haste, bandeira e SMA50
    while i < len(data) - dias_apos_bandeira:
        # Haste da bandeira
        tendencia = data["Adj Close"].iloc[i-janela_haste-janela_bandeira:i-janela_bandeira].pct_change().cumsum()
        if abs(tendencia.iloc[-1]) > threshold_haste:
            # Bandeira
            bandeira_data = data["Adj Close"].iloc[i-janela_bandeira:i]
            if bandeira_data.max() - bandeira_data.min() < bandeira_data.iloc[-1] * 0.05:  # Condição de consolidação
                bandeiras.append((i - janela_bandeira, i))
                # Calcular o movimento após a bandeira
                preco_inicio = data["Adj Close"].iloc[i]
                preco_fim = data["Adj Close"].iloc[min(i + dias_apos_bandeira, len(data)-1)]
                
                # Determinar se a operação é long ou short com base na posição da bandeira em relação à SMA50
                if bandeira_data.mean() > data["SMA50"].iloc[i-janela_bandeira:i].mean():
                    # Operação de compra (long)
                    movimento_percentual = ((preco_fim - preco_inicio) / preco_inicio) * 100
                else:
                    # Operação de venda (short)
                    movimento_percentual = ((preco_inicio - preco_fim) / preco_inicio) * 100
                
                movimentos.append(movimento_percentual)
                print(f"Padrão de bandeira identificado entre {data.index[i - janela_bandeira]} e {data.index[i]}, com movimento de {movimento_percentual:.2f}% após {dias_apos_bandeira} dias.")
                i += dias_apos_bandeira  # Pular para após o fim da bandeira para evitar sobreposição
                continue
        i += 1  # Incrementar o índice se não for encontrado padrão de bandeira

    # Somar todos os movimentos e imprimir o total
    total_movimento = sum(movimentos)
    print(f"Total de movimento combinado de todas as operações: {total_movimento:.2f}%")
    return bandeiras, movimentos

bandeiras, movimentos = detectar_bandeiras(data)




[*********************100%***********************]  1 of 1 completed
Padrão de bandeira identificado entre 2010-07-20 00:00:00 e 2010-08-10 00:00:00, com movimento de -2.87% após 5 dias.
Padrão de bandeira identificado entre 2010-12-15 00:00:00 e 2011-01-06 00:00:00, com movimento de 3.58% após 5 dias.
Padrão de bandeira identificado entre 2011-03-21 00:00:00 e 2011-04-11 00:00:00, com movimento de -0.32% após 5 dias.
Padrão de bandeira identificado entre 2011-05-10 00:00:00 e 2011-06-01 00:00:00, com movimento de 3.84% após 5 dias.
Padrão de bandeira identificado entre 2011-08-23 00:00:00 e 2011-09-14 00:00:00, com movimento de 5.87% após 5 dias.
Padrão de bandeira identificado entre 2011-11-30 00:00:00 e 2011-12-21 00:00:00, com movimento de -2.19% após 5 dias.
Padrão de bandeira identificado entre 2012-05-21 00:00:00 e 2012-06-12 00:00:00, com movimento de -1.95% após 5 dias.
Padrão de bandeira identificado entre 2012-07-03 00:00:00 e 2012-07-25 00:00:00, com movimento de 5.54% após

In [175]:
# Criar o gráfico
fig = go.Figure()

# Destacar bandeiras identificadas desenhando retângulos na zona de preços
for inicio, fim in bandeiras:
    max_preco = data["Adj Close"].iloc[inicio:fim].max()
    min_preco = data["Adj Close"].iloc[inicio:fim].min()
    
    fig.add_shape(type = "rect",
                  xref = "x", yref = "y",
                  x0 = data.index[inicio], y0 = min_preco,
                  x1 = data.index[fim], y1 = max_preco,
                  line = dict(color = "green"),
                  fillcolor = "green", opacity = 0.25)
    
# Adicionar a série de preços e a média móvel de 50 dias
fig.add_trace(go.Scatter(x=data.index, y=data["Adj Close"], mode = "lines", name = "Preço de Fechamento Ajustado", line = dict(color = "blue", width = 2)))
fig.add_trace(go.Scatter(x=data.index, y=data["SMA50"], mode = "lines", name = "SMA 50 Dias", line = dict(color = "red", width = 2)))


# Configurar layout do gráfico
fig.update_layout(title = "Identificação Avançada de Padrões de Bandeira com Análise de Movimento - " + ticker, xaxis_title = "Data", yaxis_title = "Preço", showlegend = True)

fig.update_layout(height = 800, width = 1200
                  , font_color = "blue"
                  , title_font_color = "black"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(hovermode = "x unified")

# Mostrar o gráfico
fig.show()

In [176]:
# Calcular o resultado cumulativo dos movimentos
resultados_cumulativos = np.cumsum(movimentos)

# Criar um gráfico para os resultados cumulativos
fig = go.Figure()
fig.add_trace(go.Scatter(x = list(range(len(resultados_cumulativos))), y = resultados_cumulativos, mode = "lines+markers", name = "Desempenho Cumulativo"))

# Configurar layout do gráfico
fig.update_layout(title = "Desempenho Cumulativo Baseado nos Padrões de Bandeira Identificados - " + ticker, xaxis_title = "Número da Operação", yaxis_title = "Desempenho Cumulativo (%)", showlegend=True)

fig.update_layout(height = 800, width = 1200
                  , font_color = "blue"
                  , title_font_color = "black"
                  , font = dict(size = 15, color = "Black")
                 )

# Mostrar o gráfico
fig.show()